<a href="https://colab.research.google.com/github/shuvanyu/Document-Retrieval-and-Ranking/blob/main/condenser_nfcorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=65e017560ced8ba81d69fb6e5ffe7be6f0bdfff0323b2759ee54dcadce0a3e39
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [2]:
!pip install beir


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━

In [3]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 29.8 MB/s eta 0:00:00


# Testing beir

In [4]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

import logging
import pathlib, os

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

#### Download scifact.zip dataset and unzip the dataset
dataset = "nfcorpus"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join('drive', 'MyDrive',  "nlp_datashare")

/usr/local/lib/python3.9/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:

data_path = util.download_and_unzip(url, out_dir)
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"


  0%|          | 0/3633 [00:00<?, ?it/s]

In [6]:
sorted(list(corpus.keys()))[3]

'MED-1003'

# Co-condensor

In [7]:
len(corpus)

3633

In [8]:
model_path = "sentence-transformers/msmarco-bert-co-condensor"
model = models.SentenceBERT(model_path=model_path)
normalize = True


In [9]:
model = DRES(model, batch_size=16)
retriever = EvaluateRetrieval(model, score_function="dot") # or "cos_sim" for cosine similarity
results = retriever.retrieve(corpus, queries)

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Batches:   0%|          | 0/228 [00:00<?, ?it/s]

In [10]:
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

In [11]:
ndcg

{'NDCG@1': 0.43498,
 'NDCG@3': 0.37308,
 'NDCG@5': 0.34707,
 'NDCG@10': 0.31806,
 'NDCG@100': 0.28839,
 'NDCG@1000': 0.37973}

In [12]:
_map

{'MAP@1': 0.05594,
 'MAP@3': 0.0861,
 'MAP@5': 0.10042,
 'MAP@10': 0.11702,
 'MAP@100': 0.14415,
 'MAP@1000': 0.15778}

In [13]:
recall

{'Recall@1': 0.05594,
 'Recall@3': 0.09794,
 'Recall@5': 0.12224,
 'Recall@10': 0.15288,
 'Recall@100': 0.28486,
 'Recall@1000': 0.6152}

In [14]:
precision

{'P@1': 0.44892,
 'P@3': 0.34675,
 'P@5': 0.29474,
 'P@10': 0.23158,
 'P@100': 0.07241,
 'P@1000': 0.02027}

In [15]:
import random
random.seed(250)

#### Print top-k documents retrieved ####
top_k = 10

query_id, ranking_scores = random.choice(list(results.items()))


In [16]:

query_id

'PLAIN-2177'

In [17]:
len(ranking_scores)

1001

In [18]:
scores_sorted = sorted(ranking_scores.items(), key=lambda item: item[1], reverse=True)
print("Query : %s\n" % queries[query_id])

for rank in range(min(top_k, len(ranking_scores))):
    doc_id = scores_sorted[rank][0]
    # Format: Rank x: ID [Title] Body
    print("Rank %d: %s [%s] - %s\n" % (rank+1, doc_id, corpus[doc_id].get("title"), corpus[doc_id].get("text")))

Query : sulfur

Rank 1: MED-3282 [Generation of gaseous sulfur-containing compounds in tumour tissue and suppression of gas diffusion as an antitumour treatment.] - BACKGROUND AND AIMS: The mechanisms of cancer cell growth and metastasis are still not entirely understood, especially from the viewpoint of chemical reactions in tumours. Glycolytic metabolism is markedly accelerated in cancer cells, causing the accumulation of glucose (a reducing sugar) and methionine (an amino acid), which can non-enzymatically react and form carcinogenic substances. There is speculation that this reaction produces gaseous sulfur-containing compounds in tumour tissue. The aims of this study were to clarify the products in tumour and to investigate their effect on tumour proliferation. METHODS: Products formed in the reaction between glucose and methionine or its metabolites were analysed in vitro using gas chromatography. Flatus samples from patients with colon cancer and exhaled air samples from patient

# Random Query

In [ ]:

import torch

In [ ]:
if normalize:
    corpus_embs = model.encode_corpus(reduced_corpus, batch_size=128, convert_to_tensor=True, normalize_embeddings=True)
else:
    corpus_embs = model.encode_corpus(reduced_corpus, batch_size=128, convert_to_tensor=True)

In [ ]:
query = ''

In [ ]:
start = datetime.datetime.now()
if normalize:
    query_emb = model.encode_queries([query], batch_size=1, convert_to_tensor=True, normalize_embeddings=True, show_progress_bar=False)
else:
    query_emb = model.encode_queries([query], batch_size=1, convert_to_tensor=True, show_progress_bar=False)

#### Dot product for normalized embeddings is equal to cosine similarity
sim_scores = utils.dot_score(query_emb, corpus_embs)
sim_scores_top_k_values, sim_scores_top_k_idx = torch.topk(sim_scores, 10, dim=1, largest=True, sorted=True)
end = datetime.datetime.now()

#### Measuring time taken in ms (milliseconds)
time_taken = (end - start)
time_taken = time_taken.total_seconds() * 1000
time_taken_all[query_id] = time_taken
print("{}: {} {:.2f}ms".format(query_id, query, time_taken))